In [9]:
## Extract info from OSM files

In [10]:
import hashlib
import json
import pandas as pd

In [11]:
names = pd.read_csv('data/firstnames.csv', delimiter=';')
female_names = set(names[(names['gender'].str.contains('F')) |
                         (names['gender'] == '?') |
                         (names['gender'].str.contains('1'))]['name'])
male_names = set(names[(names['gender'].str.contains('M')) |
                       (names['gender'] == '?') |
                       (names['gender'].str.contains('1'))]['name'])

In [12]:
df = pd.read_csv('data/berlin-streets.csv', delimiter='\t', header=0, names=['oname', 'objectid', 'type', 'street'])
df.dropna(subset=['street'], inplace=True)
df['id'] = df.apply(lambda x: hashlib.sha1(x['street'].encode('utf-8')).hexdigest(), axis=1)
df.drop_duplicates(subset=['id'], inplace=True)

In [13]:
df['extracted_name'] = df.apply(lambda x: ' '.join(x['street'].split('-')[0:-1]) or None, axis=1)
df['maybe_person'] = df.apply(lambda x: True if x['extracted_name'] else None, axis=1)
df['maybe_woman'] = df.apply(lambda x: (True if set(x['extracted_name'].split(' ')).intersection(female_names)
                                        else None) if x['extracted_name'] else None, axis=1)
df['maybe_man'] = df.apply(lambda x: (True if set(x['extracted_name'].split(' ')).intersection(male_names)
                                      else None) if x['extracted_name'] else None, axis=1)
df['is_person'] = None
df['is_woman'] = None
df['is_man'] = None

json_str = json.dumps(json.loads(df.to_json(orient='records')), indent=2)
with open('streets.json', 'w') as outfile:
        outfile.write(json_str)

In [14]:
## Extract street lists alphabetically

In [15]:
import locale
# locale.getlocale()
# help(locale)
locale.setlocale(locale.LC_ALL, 'de_DE.UTF-8')

'de_DE.UTF-8'

In [32]:
df.head()

,oname,objectid,type,street,id,extracted_name,maybe_person,maybe_woman,maybe_man,is_person,is_woman,is_man
0,way,4045220,residential,Hönower Straße,2d42111e7ff9690b98b9407284cb09ac0a722801,None,None,None,None,None,None,None
1,way,4045223,residential,Gundelfinger Straße,fc3ddb9e22ddadd545b15838b58b436379f50cff,None,None,None,None,None,None,None
2,way,4045243,primary,Frankfurter Allee,517751ac00ccce6f88ff73afc3102f1e12572d03,None,None,None,None,None,None,None
3,way,4045655,residential,Am Wiesengrund,b9e243214d74ad30ed84d59e3ac28387f48821de,None,None,None,None,None,None,None
4,way,4045656,secondary,Hauptstraße,2fbc04a0a21c7042c4105df0ad79ce16c3d156e1,None,None,None,None,None,None,None


In [85]:
from IPython.display import display, Markdown
initial_chars = list(filter(lambda x: x.isalpha(),
                            sorted(set([n[0].upper() for n in df['street'].dropna()]),
                                   key=locale.strxfrm)))

# [(letter, len(df[df['street'].str.startswith(letter)])) for letter in initial_chars]

def gen_markdown_table(c):


    def border(row):
        return '|' + row + '|\n'
    
    
    def rpad(val, target_len):
        return val + ' ' * (1 + target_len - len(str(val)))
        

    fields = ['extracted_name',
              'street',
              'type',
              'maybe_person',
              'maybe_woman',
              'maybe_man',
              'is_person',
              'is_woman',
              'is_man']
    length = {'street': 0,
              'extracted_name': 0,
              'type': 0,
              'maybe_person': 12,
              'maybe_woman': 11,
              'maybe_man': 9,
              'is_person': 9,
              'is_woman': 8,
              'is_man': 7
             }
    subset = df[df['street'].str.startswith(c)].sort_values(by='street', inplace=False)
    for f in fields:
        length[f] = max((length[f],
                         max(subset.apply(lambda x: len(str(x[f])), axis=1))
                        )
                       )
    md = border('|'.join([' ' + rpad(f, length[f]) for f in fields]))
    md += border('|'.join(['-' * (length[f] + 2) for f in fields]))
    for idx, row in subset.iterrows():
        md += border('|'.join([' ' + rpad(str(row[f]), length[f]) if row[f] else '' for f in fields]))
    
    display(Markdown(md))
    print(md)

gen_markdown_table('A')

| extracted_name                | street                              | type           | maybe_person | maybe_woman | maybe_man | is_person | is_woman | is_man  |
|-------------------------------|-------------------------------------|----------------|--------------|-------------|-----------|-----------|----------|---------|
| A                             | A-Weg                               | footway        | True         ||||||
| AEG                           | AEG-Siedlung Heimat                 | residential    | True         ||||||
| AEG                           | AEG-Straße                          | living_street  | True         ||||||
|| AS Bucher Straße                    | motorway_link  |||||||
|| AS Schloßstraße                     | motorway_link  |||||||
|| AS Schönerlinder Straße             | motorway_link  |||||||
|| AS Wolfensteindamm                  | secondary      |||||||
|| AVUS                                | motorway_link  |||||||
|| AVUS Nordkurve                      | service        |||||||
| AVUS                          | AVUS-Innenraum                      | service        | True         ||||||
|| Aachener Straße                     | residential    |||||||
| Aagaard                       | Aagaard-Tunnel                      | footway        | True         ||||||
|| Aalemannkanalbrücke                 | footway        |||||||
|| Aalemannufer                        | tertiary       |||||||
|| Aalesunder Straße                   | residential    |||||||
|| Aalstieg                            | residential    |||||||
|| Aalweg                              | track          |||||||
|| Aarauer Straße                      | residential    |||||||
|| Aarberger Straße                    | residential    |||||||
|| Abajstraße                          | residential    |||||||
|| Abbe Lemire Weg                     | footway        |||||||
|| Abbestraße                          | residential    |||||||
|| Abendrotweg                         | residential    |||||||
| Abram Joffe                   | Abram-Joffe-Straße                  | unclassified   | True         || True      ||||
|| Abteibrücke                         | footway        |||||||
|| Abtstraße                           | residential    |||||||
|| Abtweilerstraße                     | residential    |||||||
|| Achardstraße                        | residential    |||||||
|| Achenbachstraße                     | residential    |||||||
|| Achenseeweg                         | residential    |||||||
|| Achillesstraße                      | tertiary       |||||||
| Achsenkreuz Nord Süd/West Ost | Achsenkreuz Nord-Süd/West-Ost-Achse | disused        | True         ||||||
|| Achtermannstraße                    | residential    |||||||
|| Achtrutenberg                       | residential    |||||||
|| Ackerplanweg                        | residential    |||||||
|| Ackerrain                           | track          |||||||
|| Ackerstraße                         | residential    |||||||
|| Adalbertstraße                      | residential    |||||||
| Adam von Trott                | Adam-von-Trott-Straße               | unclassified   | True         || True      ||||
|| Adamstraße                          | tertiary       |||||||
| Adele Sandrock                | Adele-Sandrock-Straße               | residential    | True         | True        |||||
| Adele Schreiber Krieger       | Adele-Schreiber-Krieger-Straße      | residential    | True         | True        |||||
| Adelheid Poninska             | Adelheid-Poninska-Straße            | residential    | True         | True        |||||
|| Adelheidallee                       | service        |||||||
|| Adersleber Weg                      | residential    |||||||
|| Adickesstraße                       | residential    |||||||
|| Adlerbrücke                         | footway        |||||||
|| Adlergestell                        | tertiary       |||||||
|| Adlerhorst                          | service        |||||||
|| Adlershofer Straße                  | residential    |||||||
|| Adlerstraße                         | residential    |||||||
|| Adlerweg                            | footway        |||||||
|| Admiralbrücke                       | living_street  |||||||
|| Admiralstraße                       | living_street  |||||||
| Adolf Martens                 | Adolf-Martens-Straße                | residential    | True         || True      ||||
| Adolf Menzel                  | Adolf-Menzel-Straße                 | residential    | True         || True      ||||
| Adolf Scheidt                 | Adolf-Scheidt-Platz                 | residential    | True         || True      ||||
|| Adolfstraße                         | residential    |||||||
|| Adorfer Straße                      | residential    |||||||
|| Advokatensteig                      | path           |||||||
|| Aegirstraße                         | residential    |||||||
|| Affensteinweg                       | track          |||||||
|| Afrikanische Straße                 | tertiary       |||||||
|| Agathenweg                          | residential    |||||||
|| Agavensteig                         | track          |||||||
| Agnes Hacker                  | Agnes-Hacker-Straße                 | service        | True         | True        |||||
| Agnes Kraus                   | Agnes-Kraus-Weg                     | path           | True         | True        |||||
| Agnes Straub                  | Agnes-Straub-Weg                    | residential    | True         | True        |||||
| Agnes Wabnitz                 | Agnes-Wabnitz-Straße                | residential    | True         | True        |||||
| Agnes von Zahn Harnack        | Agnes-von-Zahn-Harnack-Straße       | living_street  | True         | True        |||||
|| Agnethaweg                          | residential    |||||||
|| Agricolastraße                      | residential    |||||||
|| Ahlbecker Straße                    | residential    |||||||
|| Ahlbeerensteig                      | residential    |||||||
|| Ahlener Weg                         | residential    |||||||
|| Ahornallee                          | residential    |||||||
|| Ahornsteig                          | footway        |||||||
|| Ahornstraße                         | residential    |||||||
|| Ahornweg                            | path           |||||||
|| Ahrensdorfer Straße                 | residential    |||||||
|| Ahrensfelder Chaussee               | secondary      |||||||
|| Ahrensfelder Platz                  | service        |||||||
|| Ahrensfelder Weg                    | track          |||||||
|| Ahrenshooper Straße                 | residential    |||||||
|| Ahrenshooper Zeile                  | residential    |||||||
|| Ahrweilerstraße                     | residential    |||||||
|| Aiblinger Weg                       | residential    |||||||
|| Aidastraße                          | residential    |||||||
|| Akazienallee                        | residential    |||||||
|| Akazienhof                          | residential    |||||||
|| Akazienstraße                       | residential    |||||||
|| Akazienweg                          | residential    |||||||
|| Akeleiweg                           | service        |||||||
|| Akkordeonweg                        | residential    |||||||
|| Aladinweg                           | track          |||||||
|| Alarichstraße                       | tertiary       |||||||
|| Alaskaweg                           | footway        |||||||
|| Albanstraße                         | residential    |||||||
|| Alberichstraße                      | cycleway       |||||||
|| Albersweilerweg                     | residential    |||||||
| Albert Brodersen              | Albert-Brodersen-Allee              | footway        | True         || True      ||||
| Albert Einstein               | Albert-Einstein-Straße              | service        | True         || True      ||||
| Albert Hößler                 | Albert-Hößler-Straße                | residential    | True         || True      ||||
| Albert Kuntz                  | Albert-Kuntz-Straße                 | residential    | True         || True      ||||
| Albert Schweitzer             | Albert-Schweitzer-Platz             | service        | True         || True      ||||
| Albert Schweitzer             | Albert-Schweitzer-Straße            | service        | True         || True      ||||
|| Albertinenstraße                    | residential    |||||||
|| Albertstraße                        | residential    |||||||
|| Albestraße                          | residential    |||||||
|| Albiger Weg                         | residential    |||||||
|| Alboinplatz                         | tertiary       |||||||
|| Alboinstraße                        | tertiary       |||||||
| Albrecht Achilles             | Albrecht-Achilles-Straße            | residential    | True         || True      ||||
| Albrecht Berblinger           | Albrecht-Berblinger-Straße          | residential    | True         || True      ||||
| Albrecht Dürer                | Albrecht-Dürer-Straße               | residential    | True         || True      ||||
| Albrecht Thaer                | Albrecht-Thaer-Weg                  | residential    | True         || True      ||||
|| Albrechts Teerofen                  | residential    |||||||
|| Albrechtshofer Weg                  | residential    |||||||
|| Albrechtstraße                      | residential    |||||||
|| Albtalweg                           | residential    |||||||
|| Albulaweg                           | living_street  |||||||
|| Alemannenallee                      | residential    |||||||
|| Alemannenbrücke                     | residential    |||||||
|| Alemannenstraße                     | residential    |||||||
|| Alerichstraße                       | track          |||||||
| Alex Wedding                  | Alex-Wedding-Straße                 | residential    | True         | True        | True      ||||
| Alexander Meißner             | Alexander-Meißner-Straße            | proposed       | True         || True      ||||
| Alexander von Humboldt        | Alexander-von-Humboldt-Weg          | path           | True         || True      ||||
|| Alexandersteg                       | service        |||||||
|| Alexanderstraße                     | primary        |||||||
|| Alexanderufer                       | residential    |||||||
|| Alexandrinenstraße                  | residential    |||||||
|| Alfelder Straße                     | residential    |||||||
| Alfons Loewe                  | Alfons-Loewe-Straße                 | living_street  | True         || True      ||||
|| Alfonsstraße                        | residential    |||||||
| Alfred Balen                  | Alfred-Balen-Weg                    | footway        | True         || True      ||||
| Alfred Döblin                 | Alfred-Döblin-Straße                | residential    | True         || True      ||||
| Alfred Grenander              | Alfred-Grenander-Platz              | footway        | True         || True      ||||
| Alfred Jung                   | Alfred-Jung-Straße                  | residential    | True         | True        | True      ||||
| Alfred Kowalke                | Alfred-Kowalke-Straße               | tertiary       | True         || True      ||||
| Alfred Lion                   | Alfred-Lion-Steg                    | footway        | True         || True      ||||
| Alfred Randt                  | Alfred-Randt-Straße                 | residential    | True         || True      ||||
| Alfred Rojek                  | Alfred-Rojek-Weg                    | residential    | True         || True      ||||
| Alfred Scholz                 | Alfred-Scholz-Platz                 | pedestrian     | True         || True      ||||
| Alfred Siggel                 | Alfred-Siggel-Weg                   | living_street  | True         || True      ||||
|| Alfredstraße                        | tertiary       |||||||
| Alice Archenhold              | Alice-Archenhold-Weg                | service        | True         | True        |||||
| Alice Berend                  | Alice-Berend-Straße                 | residential    | True         | True        | True      ||||
| Alice Salomon                 | Alice-Salomon-Platz                 | secondary      | True         | True        | True      ||||
| Alice und Hella Hirsch        | Alice-und-Hella-Hirsch-Ring         | residential    | True         | True        |||||
|| Alkenweg                            | residential    |||||||
|| Allee der Kosmonauten               | secondary      |||||||
|| Allendeweg                          | residential    |||||||
|| Allendorfer Weg                     | residential    |||||||
|| Allerstraße                         | residential    |||||||
|| Allerweg                            | track          |||||||
|| Allgäuer Weg                        | residential    |||||||
|| Allmendeweg                         | unclassified   |||||||
|| Allmersweg                          | residential    |||||||
|| Allée St. Exupéry                   | residential    |||||||
|| Allée du Stade                      | residential    |||||||
|| Almazeile                           | residential    |||||||
|| Almstadtstraße                      | residential    |||||||
|| Almutstraße                         | residential    |||||||
|| Almweg                              | living_street  |||||||
|| Aloisiusweg                         | footway        |||||||
|| Alpenberger Straße                  | residential    |||||||
|| Alpenhöhe                           | footway        |||||||
|| Alpenrose Weg                       | track          |||||||
|| Alpenrosenweg                       | residential    |||||||
|| Alpenveilchenweg                    | residential    |||||||
|| Alpenweg                            | footway        |||||||
|| Alpnacher Weg                       | residential    |||||||
|| Alsaceweg                           | residential    |||||||
|| Alsbacher Weg                       | living_street  |||||||
|| Alsenbrücke                         | tertiary       |||||||
|| Alsenstraße                         | residential    |||||||
|| Alsenzer Weg                        | residential    |||||||
|| Alsheimer Straße                    | residential    |||||||
|| Alsterweg                           | residential    |||||||
|| Alt Tegel                           | pedestrian     |||||||
| Alt                           | Alt-Altglienicker Brücke            | unclassified   | True         ||||||
| Alt                           | Alt-Biesdorf                        | primary        | True         ||||||
| Alt                           | Alt-Blankenburg                     | secondary      | True         ||||||
| Alt                           | Alt-Britz                           | residential    | True         ||||||
| Alt                           | Alt-Buch                            | secondary_link | True         ||||||
| Alt                           | Alt-Buckow                          | residential    | True         ||||||
| Alt                           | Alt-Friedrichsfelde                 | primary        | True         ||||||
| Alt                           | Alt-Gatow                           | secondary      | True         ||||||
| Alt                           | Alt-Heiligensee                     | secondary      | True         ||||||
| Alt                           | Alt-Hellersdorf                     | residential    | True         ||||||
| Alt                           | Alt-Hermsdorf                       | path           | True         ||||||
| Alt                           | Alt-Karow                           | secondary      | True         ||||||
| Alt                           | Alt-Kaulsdorf                       | primary        | True         ||||||
| Alt                           | Alt-Kladow                          | residential    | True         ||||||
| Alt                           | Alt-Köpenick                        | pedestrian     | True         ||||||
| Alt                           | Alt-Lankwitz                        | residential    | True         ||||||
| Alt                           | Alt-Lichtenrade                     | residential    | True         ||||||
| Alt                           | Alt-Lietzow                         | residential    | True         ||||||
| Alt                           | Alt-Lübars                          | tertiary       | True         ||||||
| Alt                           | Alt-Mahlsdorf                       | primary        | True         ||||||
| Alt                           | Alt-Mariendorf                      | primary        | True         ||||||
| Alt                           | Alt-Marienfelde                     | residential    | True         ||||||
| Alt                           | Alt-Marzahn                         | residential    | True         ||||||
| Alt                           | Alt-Moabit                          | secondary      | True         ||||||
| Alt                           | Alt-Moabit/Rathenower Straße        | platform       | True         ||||||
| Alt                           | Alt-Müggelheim                      | secondary      | True         ||||||
| Alt                           | Alt-Pichelsdorf                     | residential    | True         ||||||
| Alt                           | Alt-Reinickendorf                   | residential    | True         ||||||
| Alt                           | Alt-Rudow                           | unclassified   | True         ||||||
| Alt                           | Alt-Schmöckwitz                     | residential    | True         ||||||
| Alt                           | Alt-Schönow                         | residential    | True         ||||||
| Alt                           | Alt-Stralau                         | tertiary       | True         ||||||
| Alt                           | Alt-Tegel                           | pedestrian     | True         ||||||
| Alt                           | Alt-Tempelhof                       | tertiary       | True         ||||||
| Alt                           | Alt-Treptow                         | service        | True         ||||||
| Alt                           | Alt-Wittenau                        | tertiary       | True         ||||||
|| Altarsteinweg                       | track          |||||||
|| Altdammer Weg                       | residential    |||||||
|| Altdorfer Straße                    | residential    |||||||
|| Alte Allee                          | residential    |||||||
|| Alte Brauerei                       | service        |||||||
|| Alte Fasanerie                      | track          |||||||
|| Alte Hellersdorfer Straße           | tertiary       |||||||
|| Alte Jafféstraße                    | secondary      |||||||
|| Alte Jakobstraße                    | service        |||||||
|| Alte Kaulsdorfer Straße             | tertiary       |||||||
|| Alte Köpenicker Allee               | track          |||||||
|| Alte Leipziger Straße               | residential    |||||||
|| Alte Potsdamer Straße               | residential    |||||||
|| Alte Rhinstraße                     | residential    |||||||
|| Alte Schildower Straße              | residential    |||||||
|| Alte Schönhauser Straße             | tertiary       |||||||
|| Alte Spandauer Poststraße           | track          |||||||
|| Altenauer Weg                       | residential    |||||||
|| Altenberger Weg                     | residential    |||||||
|| Altenbraker Straße                  | residential    |||||||
|| Altenburger Allee                   | residential    |||||||
|| Altenburger Straße                  | residential    |||||||
|| Altenescher Weg                     | residential    |||||||
|| Altenhofer Straße                   | residential    |||||||
|| Altenhofer Weg                      | residential    |||||||
|| Altensteinstraße                    | tertiary       |||||||
|| Altensteinstraße/Fabeckstraße       | platform       |||||||
|| Altentreptower Straße               | secondary      |||||||
|| Alter Bernauer Heerweg              | unclassified   |||||||
|| Alter Fischerweg                    | tertiary       |||||||
|| Alter Fließweg                      | path           |||||||
|| Alter Hegemeisterweg                | path           |||||||
|| Alter Markt                         | residential    |||||||
|| Alter Postweg                       | track          |||||||
|| Alter Radelander Weg                | track          |||||||
|| Alter Schönefelder Weg              | residential    |||||||
|| Alter Segelfliegerdamm              | residential    |||||||
|| Alter Wiesenweg                     | service        |||||||
|| Altes Gaswerk Mariendorf            | unclassified   |||||||
|| Altglienicker Brücke                | secondary      |||||||
|| Altglienicker Chaussee              | secondary      |||||||
|| Altglienicker Grund                 | service        |||||||
|| Altgrabauer Straße                  | residential    |||||||
|| Althansweg                          | residential    |||||||
|| Altheider Straße                    | residential    |||||||
|| Althoffplatz                        | residential    |||||||
|| Althoffstraße                       | residential    |||||||
|| Altkanzlerstraße                    | residential    |||||||
|| Altkanzlerstraßenbrücke             | residential    |||||||
|| Altkircher Straße                   | residential    |||||||
|| Altlandsberger Platz                | footway        |||||||
|| Altmarkstraße                       | residential    |||||||
|| Altonaer Straße                     | secondary      |||||||
|| Altrader Weg                        | residential    |||||||
|| Altstädter Ring                     | secondary      |||||||
|| Altvaterstraße                      | footway        |||||||
|| Alvenslebenplatz                    | residential    |||||||
|| Alvenslebenstraße                   | residential    |||||||
|| Alwineweg                           | residential    |||||||
|| Alzeyweg                            | living_street  |||||||
|| Am Adlergestell                     | service        |||||||
|| Am Ahornweg                         | service        |||||||
|| Am Akazienweg                       | service        |||||||
|| Am Alten Fenn                       | residential    |||||||
|| Am Alten Friedhof                   | residential    |||||||
|| Am Alten Lokschuppen                | footway        |||||||
|| Am Amselsteg                        | footway        |||||||
|| Am Amtsgraben                       | footway        |||||||
|| Am Anger                            | residential    |||||||
|| Am Ansitz                           | residential    |||||||
|| Am Appelhorst                       | residential    |||||||
|| Am Ausblick                         | residential    |||||||
|| Am Bachrain                         | track          |||||||
|| Am Bahndamm                         | residential    |||||||
|| Am Bahnhof Grunewald                | residential    |||||||
|| Am Bahnhof Jungfernheide            | service        |||||||
|| Am Bahnhof Spandau                  | tertiary       |||||||
|| Am Bahnhof Westend                  | residential    |||||||
|| Am Bahnhof Wuhlheide                | service        |||||||
|| Am Baltenring                       | tertiary       |||||||
|| Am Barnim                           | service        |||||||
|| Am Bauernwäldchen                   | residential    |||||||
|| Am Bauersee                         | service        |||||||
|| Am Beelitzhof                       | residential    |||||||
|| Am Berg                             | track          |||||||
|| Am Berge                            | residential    |||||||
|| Am Berghang                         | residential    |||||||
|| Am Bergpfuhl                        | residential    |||||||
|| Am Berl                             | service        |||||||
|| Am Berlin Museum                    | living_street  |||||||
|| Am Biberbau                         | residential    |||||||
|| Am Binsengrund                      | residential    |||||||
|| Am Birkenhügel                      | residential    |||||||
|| Am Birkenknick                      | residential    |||||||
|| Am Birkenrevier                     | living_street  |||||||
|| Am Birkenwerder                     | residential    |||||||
|| Am Bogen                            | residential    |||||||
|| Am Bootshaus                        | living_street  |||||||
|| Am Borsigturm                       | unclassified   |||||||
|| Am Brandpfuhl                       | residential    |||||||
|| Am Breiten Luch                     | residential    |||||||
|| Am Bremsenwerk                      | residential    |||||||
|| Am Brendegraben                     | service        |||||||
|| Am Britzer Garten                   | living_street  |||||||
|| Am Brodersengarten                  | residential    |||||||
|| Am Bruchland                        | residential    |||||||
|| Am Brunnen                          | residential    |||||||
|| Am Buchenberg                       | residential    |||||||
|| Am Buckersberg                      | service        |||||||
|| Am Buddeplatz                       | residential    |||||||
|| Am Bullerweg                        | service        |||||||
|| Am Buschfeld                        | residential    |||||||
|| Am Bäkequell                        | residential    |||||||
|| Am Bärensprung                      | residential    |||||||
|| Am Böttcherberg                     | residential    |||||||
|| Am Bürgerpark                       | unclassified   |||||||
|| Am Carlsgarten                      | residential    |||||||
|| Am Carolinenhof                     | residential    |||||||
|| Am Cleantech Business Park          | unclassified   |||||||
|| Am Comeniusplatz                    | residential    |||||||
|| Am Containerbahnhof                 | service        |||||||
|| Am Dachsbau                         | residential    |||||||
|| Am Damm                             | residential    |||||||
|| Am Danewend                         | residential    |||||||
|| Am Dianaplatz                       | residential    |||||||
|| Am Doggelhof                        | service        |||||||
|| Am Dominikusteich                   | residential    |||||||
|| Am Donnerberg                       | residential    |||||||
|| Am Dorfanger                        | residential    |||||||
|| Am Dorfteich                        | living_street  |||||||
|| Am Dorfwald                         | residential    |||||||
|| Am Dreieck                          | footway        |||||||
|| Am Dörferweg                        | residential    |||||||
|| Am Eichenhain                       | tertiary       |||||||
|| Am Eichenquast                      | residential    |||||||
|| Am Eichgarten                       | residential    |||||||
|| Am Elsebrocken                      | residential    |||||||
|| Am Elsenbruch                       | residential    |||||||
|| Am Erlenbusch                       | residential    |||||||
|| Am Erlengrund                       | residential    |||||||
| Am Ernst Grube                | Am Ernst-Grube-Park                 | service        | True         | True        | True      ||||
|| Am Espenpfuhl                       | residential    |||||||
|| Am Eulenhorst                       | residential    |||||||
|| Am Falkenberg                       | tertiary       |||||||
|| Am Falkplatz                        | residential    |||||||
|| Am Faulen See                       | residential    |||||||
|| Am Feldberg                         | residential    |||||||
|| Am Fenn                             | residential    |||||||
|| Am Festplatz                        | service        |||||||
|| Am Festungsgraben                   | footway        |||||||
|| Am Feuchten Winkel                  | secondary      |||||||
|| Am Fichtenberg                      | residential    |||||||
|| Am Filmlager                        | living_street  |||||||
|| Am Finkenherd                       | residential    |||||||
|| Am Fischtal                         | residential    |||||||
|| Am Fliederbusch                     | residential    |||||||
|| Am Fließ                            | residential    |||||||
|| Am Flugplatz Gatow                  | residential    |||||||
|| Am Flutgraben                       | service        |||||||
|| Am Forstacker                       | residential    |||||||
|| Am Fort                             | residential    |||||||
|| Am Freibad                          | unclassified   |||||||
|| Am Friedrichshain                   | residential    |||||||
|| Am Fuchsbau                         | residential    |||||||
|| Am Fuchspaß                         | residential    |||||||
|| Am Fölzberg                         | living_street  |||||||
|| Am Gartenstadtweg                   | residential    |||||||
|| Am Gehrensee                        | residential    |||||||
|| Am Gemeindepark                     | residential    |||||||
|| Am Generalshof                      | residential    |||||||
|| Am Genossenschaftsring              | living_street  |||||||
|| Am Gewerbepark                      | residential    |||||||
|| Am Glinigk                          | residential    |||||||
|| Am Glockenturm                      | unclassified   |||||||
|| Am Goldmannpark                     | residential    |||||||
|| Am Graben                           | track          |||||||
|| Am Großen Rohrpfuhl                 | residential    |||||||
|| Am Großen Wannsee                   | residential    |||||||
|| Am Grünen Anger                     | residential    |||||||
|| Am Grünen Hof                       | residential    |||||||
|| Am Grünen Zipfel                    | residential    |||||||
|| Am Grüngürtel                       | residential    |||||||
|| Am Gutshof                          | residential    |||||||
|| Am Gutspark                         | residential    |||||||
|| Am Güterbahnhof Halensee            | service        |||||||
|| Am Hain                             | residential    |||||||
|| Am Hamburger Bahnhof                | service        |||||||
|| Am Hanffgraben                      | residential    |||||||
|| Am Hang                             | service        |||||||
|| Am Haselbusch                       | residential    |||||||
|| Am Haselluch                        | residential    |||||||
|| Am Havelgarten                      | residential    |||||||
|| Am Havelufer                        | living_street  |||||||
|| Am Hechtgraben                      | service        |||||||
|| Am Hegewinkel                       | residential    |||||||
|| Am Heideberg                        | residential    |||||||
|| Am Heidebusch                       | residential    |||||||
|| Am Heidefriedhof                    | residential    |||||||
|| Am Heidehof                         | living_street  |||||||
|| Am Heidesaum                        | residential    |||||||
|| Am Heimenstein                      | residential    |||||||
|| Am Heimhort                         | residential    |||||||
|| Am Hellespont                       | residential    |||||||
|| Am Herzbergteich                    | service        |||||||
|| Am Hirschsprung                     | residential    |||||||
|| Am Hirschwechsel                    | residential    |||||||
|| Am Hohen Feld                       | residential    |||||||
|| Am Horstenstein                     | residential    |||||||
|| Am Hügel                            | residential    |||||||
|| Am Hüllepfuhl                       | residential    |||||||
|| Am Iderfenngraben                   | residential    |||||||
|| Am Igelgrund                        | service        |||||||
|| Am Irissee                          | living_street  |||||||
|| Am Jartz                            | residential    |||||||
|| Am Johannistisch                    | residential    |||||||
|| Am Juliusturm                       | secondary      |||||||
|| Am Kahlschlag                       | residential    |||||||
|| Am Kanal                            | residential    |||||||
|| Am Kaniswall                        | service        |||||||
|| Am Karlsbad                         | service        |||||||
|| Am Karpfenpfuhl                     | residential    |||||||
|| Am Kesselpfuhl                      | residential    |||||||
|| Am Kiebitzpfuhl                     | residential    |||||||
|| Am Kiefernhang                      | residential    |||||||
|| Am Kienhorstgraben                  | footway        |||||||
|| Am Kienpfuhl                        | residential    |||||||
|| Am Kiesberg                         | residential    |||||||
|| Am Kiesteich                        | tertiary       |||||||
|| Am Kietzer Feld                     | service        |||||||
|| Am Kinderdorf                       | residential    |||||||
|| Am Kirchendreieck                   | living_street  |||||||
|| Am Kirchenland                      | residential    |||||||
|| Am Kirschgarten                     | residential    |||||||
|| Am Kladower Wäldchen                | service        |||||||
|| Am Klarpfuhl                        | residential    |||||||
|| Am Klauswerder                      | residential    |||||||
|| Am Kleinen Platz                    | residential    |||||||
|| Am Kleinen Wannsee                  | residential    |||||||
|| Am Kletterplatz                     | living_street  |||||||
|| Am Klötzgraben                      | living_street  |||||||
|| Am Koeltzepark                      | residential    |||||||
|| Am Konsulat                         | living_street  |||||||
|| Am Kornfeld                         | residential    |||||||
|| Am Kringel                          | residential    |||||||
|| Am Krug                             | footway        |||||||
|| Am Krummen Weg                      | residential    |||||||
|| Am Krusenick                        | residential    |||||||
|| Am Krähenberg                       | residential    |||||||
|| Am Krögel                           | service        |||||||
|| Am Kupfergraben                     | residential    |||||||
|| Am Kurzen Weg                       | residential    |||||||
|| Am Köllnischen Park                 | residential    |||||||
|| Am Küstergarten                     | residential    |||||||
|| Am Landeplatz                       | footway        |||||||
|| Am Landschaftspark Gatow            | unclassified   |||||||
|| Am Langen See                       | track          |||||||
|| Am Langen Weg                       | residential    |||||||
|| Am Lappjagen                        | residential    |||||||
|| Am Laubwald                         | residential    |||||||
|| Am Lehnshof                         | residential    |||||||
|| Am Leitbruch                        | footway        |||||||
|| Am Lindenbusch                      | track          |||||||
|| Am Lindenplatz                      | service        |||||||
|| Am Lindenweg                        | service        |||||||
|| Am Lokdepot                         | service        |||||||
|| Am Luchgraben                       | unclassified   |||||||
|| Am Lupinenfeld                      | residential    |||||||
|| Am Lustgarten                       | residential    |||||||
|| Am Lärchenweg                       | footway        |||||||
|| Am Löwentor                         | residential    |||||||
|| Am Lübarser Feld                    | residential    |||||||
| Am Maria Jankowski            | Am Maria-Jankowski-Park             | service        | True         | True        | True      ||||
|| Am Marienhain                       | residential    |||||||
|| Am Maselakepark                     | residential    |||||||
|| Am Meisenwinkel                     | service        |||||||
|| Am Mickelbruch                      | residential    |||||||
|| Am Moosbruch                        | residential    |||||||
|| Am Märchenbrunnen                   | service        |||||||
|| Am Müggelberg                       | residential    |||||||
|| Am Müggelpark                       | secondary      |||||||
|| Am Müggelsee                        | footway        |||||||
|| Am Mühlenberg                       | residential    |||||||
|| Am Mühlenfließ                      | service        |||||||
|| Am Mühlenteich                      | path           |||||||
|| Am Niederfeld                       | tertiary       |||||||
|| Am Nordbahnhof                      | residential    |||||||
|| Am Nordgraben                       | secondary      |||||||
|| Am Nußbaum                          | footway        |||||||
|| Am Oberbaum                         | secondary      |||||||
|| Am Oberhafen                        | unclassified   |||||||
| Am Oder Spree                 | Am Oder-Spree-Kanal                 | service        | True         | True        | True      ||||
|| Am Omnibushof                       | residential    |||||||
|| Am Orangeriepark                    | footway        |||||||
|| Am Ortsrand                         | residential    |||||||
|| Am Osrücken                         | residential    |||||||
|| Am Ostbahnhof                       | residential    |||||||
|| Am Packereigraben                   | service        |||||||
|| Am Pankepark                        | service        |||||||
|| Am Park                             | footway        |||||||
|| Am Parkplatz                        | service        |||||||
|| Am Petersberg                       | path           |||||||
|| Am Pfarracker                       | residential    |||||||
|| Am Pfingstberg                      | residential    |||||||
|| Am Pfuhl                            | residential    |||||||
|| Am Pichelssee                       | residential    |||||||
|| Am Pilz                             | residential    |||||||
|| Am Plumpengraben                    | residential    |||||||
|| Am Plänterwald                      | residential    |||||||
|| Am Poloplatz                        | residential    |||||||
|| Am Posseberg                        | unclassified   |||||||
|| Am Postbahnhof                      | residential    |||||||
|| Am Postfenn                         | tertiary       |||||||
|| Am Priesteracker                    | residential    |||||||
|| Am Priesterberg                     | residential    |||||||
|| Am Pumpwerk                         | residential    |||||||
|| Am Querschlag                       | residential    |||||||
|| Am Rain                             | service        |||||||
|| Am Rathaus                          | residential    |||||||
|| Am Rathauspark                      | residential    |||||||
|| Am Rehwechsel                       | residential    |||||||
|| Am Rheinischen Viertel              | residential    |||||||
|| Am Ried                             | residential    |||||||
|| Am Ritterholz                       | residential    |||||||
|| Am Rodelberg                        | residential    |||||||
|| Am Rohrbusch                        | service        |||||||
|| Am Rohrgarten                       | living_street  |||||||
|| Am Rollberg                         | residential    |||||||
|| Am Rosenanger                       | residential    |||||||
|| Am Roseneck                         | service        |||||||
|| Am Rosengarten                      | footway        |||||||
|| Am Rosenhag                         | residential    |||||||
|| Am Rosensteg                        | residential    |||||||
|| Am Roten Stein                      | service        |||||||
|| Am Rudolfplatz                      | residential    |||||||
|| Am Rudower Waldrand                 | residential    |||||||
|| Am Rundling                         | living_street  |||||||
|| Am Rupenhorn                        | residential    |||||||
|| Am Römersgrün                       | residential    |||||||
|| Am Rötepfuhl                        | residential    |||||||
|| Am Sandberg                         | track          |||||||
|| Am Sandhaus                         | residential    |||||||
|| Am Sandwerder                       | residential    |||||||
|| Am Schilf                           | living_street  |||||||
|| Am Schillertheater                  | residential    |||||||
|| Am Schlachtensee                    | residential    |||||||
|| Am Schlangengraben                  | unclassified   |||||||
|| Am Schlehdorn                       | residential    |||||||
|| Am Schlossberg                      | service        |||||||
|| Am Schloßhof                        | service        |||||||
|| Am Schloßpark                       | residential    |||||||
|| Am Schmeding                        | residential    |||||||
|| Am Schonungsberg                    | residential    |||||||
|| Am Schwanenberg                     | footway        |||||||
|| Am Schweizer Garten                 | service        |||||||
|| Am Schweizerhof                     | residential    |||||||
|| Am Schwemmhorn                      | residential    |||||||
|| Am Schäfersee                       | residential    |||||||
|| Am Schülerheim                      | residential    |||||||
|| Am Seddinsee                        | residential    |||||||
|| Am See                              | footway        |||||||
|| Am Seegraben                        | primary        |||||||
|| Am Seeschloß                        | residential    |||||||
|| Am Sonnenhügel                      | living_street  |||||||
|| Am Spandauer Wasserturm             | unclassified   |||||||
|| Am Speicher                         | service        |||||||
|| Am Spielplatz                       | service        |||||||
|| Am Sportplatz                       | unclassified   |||||||
|| Am Spreebord                        | unclassified   |||||||
|| Am Springeberg                      | residential    |||||||
|| Am Springebruch                     | living_street  |||||||
|| Am Springebruch 17 bis 27           | footway        |||||||
|| Am Springebruch 29 bis 39           | footway        |||||||
|| Am Springebruch 41 bis 51           | footway        |||||||
|| Am Springebruch 53 bis 63           | footway        |||||||
|| Am Staakener Kirchengelände         | residential    |||||||
|| Am Stadtforst                       | service        |||||||
|| Am Stadtpark                        | residential    |||||||
|| Am Stand                            | residential    |||||||
|| Am Steilhang                        | residential    |||||||
|| Am Steinberg                        | secondary      |||||||
|| Am Steinbergpark                    | living_street  |||||||
|| Am Steingarten                      | footway        |||||||
|| Am Stener Berg                      | residential    |||||||
|| Am Stichkanal                       | unclassified   |||||||
|| Am Stieggarten                      | residential    |||||||
|| Am Stieggraben                      | service        |||||||
|| Am Straßenbahnhof                   | residential    |||||||
|| Am Studio                           | service        |||||||
|| Am Sudhaus                          | living_street  |||||||
|| Am Südfeld                          | residential    |||||||
|| Am Südpark                          | residential    |||||||
|| Am Taubenschlag                     | footway        |||||||
|| Am Tegeler Hafen                    | residential    |||||||
|| Am Tegelgrund                       | residential    |||||||
|| Am Tempelgraben                     | living_street  |||||||
|| Am Tempelhofer Berg                 | residential    |||||||
|| Am Teufelsbruch                     | residential    |||||||
|| Am Theodorpark                      | residential    |||||||
|| Am Tierpark                         | secondary      |||||||
|| Am Treptower Park                   | primary        |||||||
|| Am Triftpark                        | living_street  |||||||
|| Am Unterholz                        | residential    |||||||
|| Am Viehhof                          | living_street  |||||||
|| Am Vierling                         | footway        |||||||
|| Am Vierrutenberg                    | residential    |||||||
|| Am Vierstückenpfuhl                 | residential    |||||||
|| Am Vogelherd                        | residential    |||||||
|| Am Volkspark                        | residential    |||||||
|| Am Vorwerk                          | unclassified   |||||||
|| Am Waidmannseck                     | residential    |||||||
|| Am Waldberg                         | residential    |||||||
|| Am Walde                            | residential    |||||||
|| Am Waldfriedhof                     | residential    |||||||
|| Am Waldhang                         | residential    |||||||
|| Am Waldhaus                         | residential    |||||||
|| Am Waldidyll                        | residential    |||||||
|| Am Waldkiefernweg                   | residential    |||||||
|| Am Waldpark                         | residential    |||||||
|| Am Waldrand                         | residential    |||||||
|| Am Wall                             | unclassified   |||||||
|| Am Wartenberger Luch                | living_street  |||||||
|| Am Wasserbogen                      | residential    |||||||
|| Am Wasserbunker                     | footway        |||||||
|| Am Wasserturm                       | residential    |||||||
|| Am Wasserwerk                       | residential    |||||||
|| Am Wechsel                          | residential    |||||||
|| Am Weidenbruch                      | residential    |||||||
|| Am Weidendamm                       | residential    |||||||
|| Am Weihenhorst                      | residential    |||||||
|| Am Weingarten                       | residential    |||||||
|| Am Weinhang                         | footway        |||||||
|| Am Weißen Steg                      | residential    |||||||
|| Am Werksgarten                      | living_street  |||||||
|| Am Westkreuz                        | unclassified   |||||||
|| Am Wieselbau                        | residential    |||||||
|| Am Wiesenende                       | residential    |||||||
|| Am Wiesengraben                     | residential    |||||||
|| Am Wiesengrund                      | residential    |||||||
|| Am Wiesenhang                       | track          |||||||
|| Am Wiesenhaus                       | service        |||||||
|| Am Wiesenrain                       | residential    |||||||
|| Am Wiesenweg                        | footway        |||||||
|| Am Wildbusch                        | residential    |||||||
|| Am Wildgatter                       | residential    |||||||
|| Am Windbruch                        | track          |||||||
|| Am Winkel                           | residential    |||||||
|| Am Wriezener Bahnhof                | residential    |||||||
|| Am Wuhlebogen                       | living_street  |||||||
|| Am Wuhleufer                        | unclassified   |||||||
|| Am Wäldchen                         | residential    |||||||
|| Am Zeppelinpark                     | residential    |||||||
|| Am Zeughaus                         | pedestrian     |||||||
|| Am Zirkus                           | residential    |||||||
|| Am Zwiebusch                        | unclassified   |||||||
|| Am Zwirngraben                      | pedestrian     |||||||
|| Am alten Flugplatz                  | residential    |||||||
|| Am alten Gaswerk                    | residential    |||||||
|| Am alten Malchower Graben           | footway        |||||||
|| Am kleinen Anger                    | living_street  |||||||
|| Amalienhofstraße                    | residential    |||||||
|| Amalienpark                         | residential    |||||||
|| Amalienstraße                       | residential    |||||||
|| Amandastraße                        | residential    |||||||
|| Amanlisweg                          | residential    |||||||
|| Ambacher Straße                     | service        |||||||
|| Amberbaumallee                      | pedestrian     |||||||
|| Amboßweg                            | residential    |||||||
|| Ameisenweg                          | residential    |||||||
| Amelia Earhart                | Amelia-Earhart-Straße               | living_street  | True         | True        |||||
| Amelie Beese                  | Amelie-Beese-Zeile                  | living_street  | True         | True        |||||
|| Amelieweg                           | residential    |||||||
|| Amendestraße                        | residential    |||||||
|| Amfortasweg                         | residential    |||||||
|| Ammerseestraße                      | residential    |||||||
|| Amorbacher Weg                      | residential    |||||||
|| Amorstraße                          | residential    |||||||
|| Amorweg                             | footway        |||||||
|| Ampferweg                           | residential    |||||||
|| Amrumer Straße                      | secondary      |||||||
|| Amsel Weg                           | track          |||||||
|| Amselgasse                          | service        |||||||
|| Amselgrund                          | residential    |||||||
|| Amselhainer Weg                     | residential    |||||||
|| Amselpfad                           | footway        |||||||
|| Amselsteg                           | track          |||||||
|| Amselsteig                          | footway        |||||||
|| Amselstraße                         | residential    |||||||
|| Amselweg                            | footway        |||||||
|| Amsterdamer Straße                  | living_street  |||||||
|| Amtsgerichtsplatz                   | tertiary       |||||||
|| Amtsstraße                          | residential    |||||||
|| An den Achterhöfen                  | residential    |||||||
|| An den Berggärten                   | residential    |||||||
|| An den Bänken                       | residential    |||||||
|| An den Eldenaer Höfen               | living_street  |||||||
|| An den Feldern                      | living_street  |||||||
|| An den Feldtmanngärten              | residential    |||||||
|| An den Fließtalhöfen                | service        |||||||
|| An den Freiheitswiesen              | unclassified   |||||||
|| An den Haselbüschen                 | service        |||||||
|| An den Hubertshäusern               | residential    |||||||
|| An den Klostergärten                | service        |||||||
|| An den Knabenhäusern                | service        |||||||
|| An den Parkbäumen                   | residential    |||||||
|| An den Rohrbruchwiesen              | residential    |||||||
|| An den Siedlergärten                | residential    |||||||
|| An den Treptowers                   | primary        |||||||
|| An den Zingerwiesen                 | residential    |||||||
|| An der Apostelkirche                | residential    |||||||
|| An der Aussicht                     | footway        |||||||
|| An der Bahn                         | track          |||||||
|| An der Bastion                      | residential    |||||||
|| An der Brauerei                     | residential    |||||||
|| An der Buche                        | residential    |||||||
|| An der Bucht                        | residential    |||||||
|| An der Dorfkirche                   | residential    |||||||
|| An der Felgenlake                   | residential    |||||||
|| An der Filmfabrik                   | living_street  |||||||
|| An der Fließwiese                   | residential    |||||||
|| An der Gatower Heide                | residential    |||||||
|| An der Hansekogge                   | footway        |||||||
|| An der Hasenfurt                    | residential    |||||||
|| An der Havelspitze                  | footway        |||||||
|| An der Heide                        | residential    |||||||
|| An der Heilandsweide                | residential    |||||||
|| An der Himmelswiese                 | residential    |||||||
|| An der Industriebahn                | residential    |||||||
|| An der Kappe                        | residential    |||||||
|| An der Karlshorster Heide           | residential    |||||||
|| An der Karolinenhöhe                | residential    |||||||
|| An der Kieler Brücke                | residential    |||||||
|| An der Kolonnade                    | residential    |||||||
|| An der Kommandantur                 | residential    |||||||
|| An der Koppel                       | residential    |||||||
|| An der Kremmender Bahn              | footway        |||||||
|| An der Krummen Lake                 | residential    |||||||
|| An der Krähenheide                  | residential    |||||||
|| An der Laake                        | residential    |||||||
|| An der Malche                       | cycleway       |||||||
|| An der Margaretenhöhe               | residential    |||||||
|| An der Michaelbrücke                | tertiary       |||||||
|| An der Mäckeritzbrücke              | residential    |||||||
|| An der Mühle                        | unclassified   |||||||
|| An der Nachtbucht                   | residential    |||||||
|| An der Neumark                      | construction   |||||||
|| An der Oberrealschule               | residential    |||||||
|| An der Obstwiese                    | living_street  |||||||
|| An der Ostbahn                      | residential    |||||||
|| An der Priesterkoppel               | residential    |||||||
|| An der Putlitzbrücke                | secondary      |||||||
|| An der Rehwiese                     | residential    |||||||
|| An der Ribishecke                   | footway        |||||||
|| An der Rudower Höhe                 | residential    |||||||
|| An der Schillingbrücke              | secondary      |||||||
|| An der Schneise                     | residential    |||||||
|| An der Schule                       | residential    |||||||
|| An der Schutzbepflanzung            | path           |||||||
|| An der Schäferei                    | residential    |||||||
|| An der Spandauer Brücke             | tertiary       |||||||
|| An der Spielwiese                   | footway        |||||||
|| An der Spitze                       | unclassified   |||||||
|| An der Spreeschanze                 | unclassified   |||||||
|| An der Straße                       | path           |||||||
|| An der Trainierbahn                 | track          |||||||
|| An der Tränke                       | residential    |||||||
|| An der Urania                       | primary        |||||||
|| An der Vogelweide                   | living_street  |||||||
|| An der Wasserstadt                  | path           |||||||
|| An der Werderlake                   | residential    |||||||
|| An der Wildbahn                     | residential    |||||||
|| An der Wuhle                        | track          |||||||
|| An der Wuhlheide                    | secondary      |||||||
|| Ancillonweg                         | residential    |||||||
|| Andanteweg                          | residential    |||||||
|| Andenzeisigweg                      | path           |||||||
|| Andernacher Straße                  | residential    |||||||
|| Andersen Weg                        | footway        |||||||
|| Andersenstraße                      | residential    |||||||
|| Andersenweg                         | footway        |||||||
|| Andersonweg                         | path           |||||||
|| Andlauer Weg                        | residential    |||||||
|| Andornsteig                         | residential    |||||||
| Andreas Hermes                | Andreas-Hermes-Siedlung             | service        | True         || True      ||||
|| Andreasberger Straße                | residential    |||||||
|| Andreasstraße                       | tertiary       |||||||
|| Andréezeile                         | residential    |||||||
|| Anemonensteig                       | track          |||||||
|| Anemonenstraße                      | residential    |||||||
|| Angelikaweg                         | residential    |||||||
|| Angerburger Allee                   | tertiary       |||||||
|| Angermünder Straße                  | residential    |||||||
|| Angersbacher Pfad                   | residential    |||||||
|| Angersteinweg                       | service        |||||||
|| Angerweg                            | residential    |||||||
|| Anglersiedlung                      | living_street  |||||||
|| Anhalter Steg                       | footway        |||||||
|| Anhalter Straße                     | tertiary       |||||||
|| Anhaltiner Weg                      | residential    |||||||
|| Anhaltinerstraße                    | residential    |||||||
|| Anisweg                             | track          |||||||
|| Ankerweg                            | footway        |||||||
|| Anklamer Straße                     | residential    |||||||
|| Ankogelweg                          | living_street  |||||||
|| Anlieferung                         | service        |||||||
|| Anlieferung 2                       | service        |||||||
|| Anliegerstraße                      | service        |||||||
| Anna Bruseberg                | Anna-Bruseberg-Straße               | track          | True         | True        |||||
| Anna Ebermann                 | Anna-Ebermann-Straße                | residential    | True         | True        |||||
| Anna Louisa Karsch            | Anna-Louisa-Karsch-Straße           | residential    | True         | True        |||||
| Anna Mackenroth               | Anna-Mackenroth-Weg                 | living_street  | True         | True        |||||
| Anna Maria Müller             | Anna-Maria-Müller-Straße            | living_street  | True         | True        | True      ||||
| Anna Nemitz                   | Anna-Nemitz-Weg                     | footway        | True         | True        |||||
| Anna Seghers                  | Anna-Seghers-Straße                 | residential    | True         | True        |||||
| Anna Siemsen                  | Anna-Siemsen-Weg                    | residential    | True         | True        |||||
|| Annaburger Straße                   | residential    |||||||
|| Annastraße                          | residential    |||||||
| Anne Frank                    | Anne-Frank-Straße                   | residential    | True         | True        | True      ||||
| Anneliese und Georg Groscurth | Anneliese-und-Georg-Groscurth-Platz | residential    | True         | True        | True      ||||
|| Annemariestraße                     | residential    |||||||
|| Annenallee                          | residential    |||||||
|| Annenstraße                         | tertiary       |||||||
|| Annenweg                            | footway        |||||||
| Anni Frid                     | Anni-Frid-Weg                       | residential    | True         | True        |||||
|| Annweilerweg                        | residential    |||||||
|| Ansbacher Straße                    | residential    |||||||
|| Anschützweg                         | residential    |||||||
|| Anselmstraße                        | residential    |||||||
|| Ansgarstraße                        | residential    |||||||
| Anton Saefkow                 | Anton-Saefkow-Platz                 | residential    | True         || True      ||||
| Anton Saefkow                 | Anton-Saefkow-Straße                | residential    | True         || True      ||||
| Anton Webern                  | Anton-Webern-Weg                    | residential    | True         || True      ||||
| Anton von Werner              | Anton-von-Werner-Straße             | residential    | True         || True      ||||
|| Antonienstraße                      | secondary      |||||||
|| Antoniuskirchstraße                 | residential    |||||||
|| Antonplatz                          | pedestrian     |||||||
|| Antonstraße                         | residential    |||||||
|| Antonyplatz                         | footway        |||||||
|| Antwerpener Straße                  | residential    |||||||
|| Anzengruberstraße                   | residential    |||||||
|| Apfelalle                           | track          |||||||
|| Apfelallee                          | service        |||||||
|| Apfelblütenweg                      | residential    |||||||
|| Apfelweg                            | track          |||||||
|| Apfelwicklerstraße                  | living_street  |||||||
|| Apoldaer Straße                     | residential    |||||||
|| Apollofalterallee                   | residential    |||||||
|| Apollostraße                        | residential    |||||||
| Apostel Paulus                | Apostel-Paulus-Straße               | residential    | True         || True      ||||
|| Appelbacher Weg                     | living_street  |||||||
|| Appenzeller Straße                  | tertiary       |||||||
|| Aprikosenallee                      | track          |||||||
|| Aprikosensteig                      | service        |||||||
|| Aprikosenweg                        | track          |||||||
|| Arabisweg                           | residential    |||||||
|| Aralienweg                          | residential    |||||||
|| Arberstraße                         | residential    |||||||
|| Archenholdstraße                    | residential    |||||||
|| Archibaldweg                        | residential    |||||||
|| Archivstraße                        | residential    |||||||
|| Arcostraße                          | residential    |||||||
|| Arendsweg                           | residential    |||||||
|| Arenholzsteig                       | residential    |||||||
|| Argenauer Straße                    | residential    |||||||
|| Argentinische Allee                 | secondary      |||||||
|| Argoallee                           | residential    |||||||
|| Argonnenweg                         | residential    |||||||
|| Ariadnestraße                       | residential    |||||||
| Aristide Briand               | Aristide-Briand-Brücke              | residential    | True         || True      ||||
|| Aristotelessteig                    | service        |||||||
|| Arkenberger Damm                    | unclassified   |||||||
|| Arkonaplatz                         | pedestrian     |||||||
|| Arkonastraße                        | residential    |||||||
|| Arkonaweg                           | track          |||||||
|| Armbrustweg                         | residential    |||||||
|| Armenische Straße                   | residential    |||||||
|| Arminiusstraße                      | residential    |||||||
|| Arndtplatz                          | residential    |||||||
|| Arndtstraße                         | residential    |||||||
|| Arneburger Straße                   | residential    |||||||
|| Arnfriedstraße                      | residential    |||||||
|| Arnfriedweg                         | residential    |||||||
|| Arnheidstraße                       | residential    |||||||
|| Arnikaweg                           | residential    |||||||
|| Arnimallee                          | residential    |||||||
|| Arnimstraße                         | residential    |||||||
| Arno Holz                     | Arno-Holz-Straße                    | residential    | True         || True      ||||
| Arno Philippsthal             | Arno-Philippsthal-Straße            | residential    | True         || True      ||||
| Arnold Knoblauch              | Arnold-Knoblauch-Ring               | residential    | True         || True      ||||
| Arnold Zweig                  | Arnold-Zweig-Straße                 | residential    | True         || True      ||||
|| Arnouxstraße                        | residential    |||||||
|| Arnsberger Straße                   | residential    |||||||
|| Arnsteinweg                         | track          |||||||
|| Arnstädter Straße                   | residential    |||||||
|| Arnulfstraße                        | tertiary       |||||||
|| Aronsstraße                         | residential    |||||||
|| Aroser Allee                        | secondary      |||||||
|| Artemisstraße                       | residential    |||||||
| Arthur Friedrich              | Arthur-Friedrich-Weg                | path           | True         || True      ||||
| Arthur Müller                 | Arthur-Müller-Straße                | living_street  | True         || True      ||||
| Arthur Weisbrodt              | Arthur-Weisbrodt-Straße             | residential    | True         || True      ||||
|| Arturweg                            | residential    |||||||
|| Artuswall                           | residential    |||||||
|| Arysallee                           | residential    |||||||
|| Asbestweg                           | residential    |||||||
|| Aschaffenburger Straße              | residential    |||||||
|| Ascheberger Weg                     | residential    |||||||
|| Aschenbrödelstraße                  | residential    |||||||
|| Aschenbrödelweg                     | track          |||||||
|| Aschenputtelweg                     | footway        |||||||
|| Ascherslebener Weg                  | residential    |||||||
|| Asgardstraße                        | residential    |||||||
|| Ashdodstraße                        | residential    |||||||
|| Askaloner Weg                       | residential    |||||||
|| Askanierring                        | residential    |||||||
|| Askanischer Platz                   | residential    |||||||
|| Asnièresstraße                      | residential    |||||||
|| Aspenweg                            | residential    |||||||
|| Asseburgpfad                        | residential    |||||||
| Asta Nielsen                  | Asta-Nielsen-Straße                 | residential    | True         | True        |||||
|| Asterngasse                         | footway        |||||||
|| Asternplatz                         | residential    |||||||
|| Asternsteg                          | path           |||||||
|| Asternsteig                         | track          |||||||
|| Asternweg                           | footway        |||||||
|| Asterweg                            | footway        |||||||
|| Astridstraße                        | residential    |||||||
|| Atlantisallee                       | service        |||||||
|| Atlantisring                        | living_street  |||||||
|| Attendorner Weg                     | residential    |||||||
|| Attilagarten                        | footway        |||||||
|| Attilaplatz                         | tertiary       |||||||
|| Attilastraße                        | secondary      |||||||
|| Attinghausenweg                     | residential    |||||||
|| Atzpodienstraße                     | tertiary       |||||||
|| Auber Steig                         | residential    |||||||
|| Aubertstraße                        | residential    |||||||
|| Auerbacher Ring                     | residential    |||||||
|| Auerbachstraße                      | residential    |||||||
|| Auerhahnbalz                        | residential    |||||||
|| Auersbergstraße                     | residential    |||||||
|| Auerstraße                          | residential    |||||||
|| Auerswaldstraße                     | residential    |||||||
|| Auf dem Grat                        | residential    |||||||
|| Auf dem Mühlenberg                  | living_street  |||||||
|| Auf der Höh                         | residential    |||||||
|| Auf der Planweide                   | residential    |||||||
|| Auffacher Weg                       | living_street  |||||||
|| Aufzug Gleis 1+3                    | elevator       |||||||
|| Aufzug Gleis 2+4                    | elevator       |||||||
|| Augenfalterstraße                   | residential    |||||||
|| Augsburger Platz                    | residential    |||||||
|| Augsburger Straße                   | residential    |||||||
| August Bebel                  | August-Bebel-Straße                 | track          | True         || True      ||||
| August Euler                  | August-Euler-Zeile                  | living_street  | True         || True      ||||
| August Froehlich              | August-Froehlich-Straße             | tertiary       | True         || True      ||||
| August Lindemann              | August-Lindemann-Straße             | residential    | True         || True      ||||
| August Siebke                 | August-Siebke-Straße                | service        | True         || True      ||||
|| Augustaplatz                        | residential    |||||||
|| Augustastraße                       | residential    |||||||
|| Augustaufer                         | path           |||||||
| Auguste Hauschner             | Auguste-Hauschner-Straße            | living_street  | True         | True        | True      ||||
| Auguste Piccard               | Auguste-Piccard-Straße              | residential    | True         | True        | True      ||||
| Auguste Viktoria              | Auguste-Viktoria-Allee              | residential    | True         | True        | True      ||||
| Auguste Viktoria              | Auguste-Viktoria-Straße             | residential    | True         | True        | True      ||||
|| Augustenburger Platz                | service        |||||||
|| Auguststraße                        | residential    |||||||
|| Aumetzer Weg                        | residential    |||||||
|| Aumühler Straße                     | residential    |||||||
|| Auraser Weg                         | residential    |||||||
|| Aurikelweg                          | residential    |||||||
|| Aurorafalterweg                     | residential    |||||||
|| Ausbau Mühle                        | primary        |||||||
|| Ausfahrt Parkhaus                   | service        |||||||
|| Ausfahrt Tor 9                      | service        |||||||
|| Ausgang Richtung Riemeisterstraße   | steps          |||||||
|| Aussenweg                           | footway        |||||||
|| Aussichtsberg                       | footway        |||||||
|| Autoverladestelle Wannsee           | service        |||||||
|| Außenweg                            | residential    |||||||
|| Avenue Charles de Gaulle            | residential    |||||||
|| Avenue Jean Mermoz                  | residential    |||||||
| Axel Springer                 | Axel-Springer-Straße                | secondary      | True         || True      ||||
|| Axenstraße                          | residential    |||||||
|| Azaleenstraße                       | residential    |||||||
|| Azaleenweg                          | footway        |||||||
|| Aßmannshauser Straße                | residential    |||||||
|| Aßmannstraße                        | service        |||||||


| extracted_name                | street                              | type           | maybe_person | maybe_woman | maybe_man | is_person | is_woman | is_man  |
|-------------------------------|-------------------------------------|----------------|--------------|-------------|-----------|-----------|----------|---------|
| A                             | A-Weg                               | footway        | True         ||||||
| AEG                           | AEG-Siedlung Heimat                 | residential    | True         ||||||
| AEG                           | AEG-Straße                          | living_street  | True         ||||||
|| AS Bucher Straße                    | motorway_link  |||||||
|| AS Schloßstraße                     | motorway_link  |||||||
|| AS Schönerlinder Straße             | motorway_link  |||||||
|| AS Wolfensteindamm                  | secondary      |||||||
|| AVUS                                | motorway_link  |||||||
|| AVUS Nordkurve          

In [21]:
df[df['street'].str.startswith('7')]

,oname,objectid,type,street,id,extracted_name,maybe_person,maybe_woman,maybe_man,is_person,is_woman,is_man
5450,way,4808092,service,7 Meter Weg,2b265e3ab283f9229561c9298e25721067465851,None,None,None,None,None,None,None
74350,way,182821475,path,7,902ba3cda1883801594b6e1b452790cc53948fda,None,None,None,None,None,None,None


In [8]:
streets.to_json('streets.json',orient='records')

NameError: name 'streets' is not defined

In [9]:
## From here we'll work on extracting street nodes
## This point and below is a WIP.

In [43]:
import json
import numpy as np
from scipy.spatial import ConvexHull
import wikipedia

In [44]:
street_nodes = df[df['type'].isin(['primary', 'secondary', 'tertiary'
                                   'residential', 'living_street', 'unclassified',
                                   'trunk', 'motorway', 'pedestrian', 'cycleway'])][['id', 'type', 'node', 'lat', 'lon']].groupby(by=['id'], axis=0)

In [45]:
# json_array = street_nodes.get_group('a65f18281e38ff7a43eb605e2a06e86c6649a337')[['lat', 'lon']].to_json(orient='values')
points = street_nodes.get_group('a65f18281e38ff7a43eb605e2a06e86c6649a337')[['lat', 'lon']].values
# points = json.loads(json_array)
# hull = ConvexHull(points)
# a = [points[x] for x in hull.vertices]
# a.append(points[hull.vertices[0]])
# pd.Series(a).to_json(orient='values')

In [46]:
street_nodes.get_group('a65f18281e38ff7a43eb605e2a06e86c6649a337')

,id,type,node,lat,lon
25640,a65f18281e38ff7a43eb605e2a06e86c6649a337,primary,1000000004436950,52.510231,13.284216
25687,a65f18281e38ff7a43eb605e2a06e86c6649a337,primary,1000000004446503,52.510527,13.291367
31299,a65f18281e38ff7a43eb605e2a06e86c6649a337,primary,1000000052578939,52.509935,13.282075
31300,a65f18281e38ff7a43eb605e2a06e86c6649a337,primary,1000000052578940,52.510090,13.282054
33475,a65f18281e38ff7a43eb605e2a06e86c6649a337,primary,1000000151821673,52.509415,13.274061
34373,a65f18281e38ff7a43eb605e2a06e86c6649a337,primary,1000000176621904,52.510716,13.291775
34374,a65f18281e38ff7a43eb605e2a06e86c6649a337,primary,1000000176621905,52.510537,13.291567
34375,a65f18281e38ff7a43eb605e2a06e86c6649a337,primary,1000000176621906,52.510549,13.291785
34376,a65f18281e38ff7a43eb605e2a06e86c6649a337,primary,1000000176621907,52.510694,13.291406
35211,a65f18281e38ff7a43eb605e2a06e86c6649a337,primary,1000000227985288,52.509600,13.274266


In [47]:
nodes = street_nodes.get_group('a65f18281e38ff7a43eb605e2a06e86c6649a337')[['lat', 'lon']].values
nodes

array([[52.5102312, 13.2842156],
       [52.5105268, 13.2913671],
       [52.5099349, 13.2820754],
       [52.51009  , 13.2820544],
       [52.5094154, 13.2740606],
       [52.5107162, 13.2917754],
       [52.5105369, 13.2915668],
       [52.5105486, 13.2917854],
       [52.5106936, 13.2914063],
       [52.5095995, 13.2742662],
       [52.5096192, 13.2741496],
       [52.5099349, 13.2820754],
       [52.5099026, 13.2815528],
       [52.5094154, 13.2740606]])

In [6]:
wikipedia.set_lang('de')
print(women.iloc[0])
wikipedia.page(wikipedia.search(women.iloc[0])[0]).title

Auguste Viktoria


'Viktoria von Dirksen'

In [124]:
results = women.apply(lambda x : wikipedia.search(x))

In [153]:
for idx, r in results[results.apply(lambda x : len(x)) < 10].iteritems():
    print({women.loc[idx]: r})

{'Hildegard Marcusson': ['Erwin Marcusson', 'Friedrichs-Waisenhaus Rummelsburg', 'Liste der Biografien/Mara–Marh', 'Liste der Straßen und Plätze in Berlin-Wilmersdorf', 'Nekrolog 1976', 'Liste der Straßen und Plätze in Berlin-Rummelsburg', 'Liste der Kulturdenkmale in Berlin-Rummelsburg']}
{'Amelie Beese': ['Amelie (Vorname)', 'Melli Beese', 'Beese', 'Liste bekannter Personen der Luftfahrt', 'Liste der Straßennamen von Frankfurt am Main/M', 'Garten der Frauen', 'Liste der Ehrengräber in Berlin', 'Liste der Straßen und Plätze in Berlin-Kladow']}
{'Rosa Reinglass': ['Liste der Straßen und Plätze in Berlin-Hakenfelde', 'Liste der Stolpersteine in Berlin-Wilhelmstadt']}
{'Anna Bruseberg': ['Liste der Straßen und Plätze in Berlin-Französisch Buchholz']}
{'Hiltrud Dudek': ['Südpark (Berlin-Rudow)', 'Liste der Straßen und Plätze in Berlin-Rudow', 'Liste der Kinos im Berliner Bezirk Neukölln']}
{'Gertrud Dorka': ['Gertrud Dorka', 'Dorka', 'Landesgeschichtliche Vereinigung für die Mark Brandenb

In [149]:
women.loc[230352]

'Anna Bruseberg'

In [7]:
df2 = pd.read_csv('data/berlin-streets.csv', delimiter='\t', header=0, names=['node', 'id', 'lat', 'long', 'country', 'city', 'street'])
df2.head()

,node,id,lat,long,country,city,street
0,node,26735749,13.322832,52.506912,DE,Berlin,Knesebeckstraße
1,node,26735759,13.318092,52.506207,DE,Berlin,Kantstraße
2,node,26735763,13.320785,52.507339,DE,Berlin,Pestalozzistraße
3,node,26756830,13.328194,52.530825,DE,Berlin,Beusselstraße
4,node,26867411,13.294379,52.501851,DE,Berlin,Heilbronner Straße


In [37]:
bystreet = df2.groupby(by=["street", "city", "country"], axis=0)

In [40]:
bystreet.groups.keys()

dict_keys([('Emil-Basdeck-Straße', 'Berlin', 'DE'), ('Haderslebener Straße', 'Berlin', 'DE'), ('Friedenfelser Straße', 'Berlin', 'DE'), ('Heinz-Bartsch-Straße', 'Berlin', 'DE'), ('Landecker Straße', 'Berlin', 'DE'), ('Bertolt-Brecht-Platz', 'Berlin', 'DE'), ('Rägeliner Straße', 'Berlin', 'DE'), ('Elisabeth-Schiemann-Straße', 'Berlin', 'DE'), ('Jasmunder Straße', 'Berlin', 'DE'), ('Beckerstraße', 'Berlin', 'DE'), ('Frischlingsteig', 'Berlin', 'DE'), ('Bölschestraße', 'Berlin', 'DE'), ('Antonplatz', 'Berlin', 'DE'), ('Schkopauer Ring', 'Berlin', 'DE'), ('Zwinglistraße', 'Berlin', 'DE'), ('Degnerstraße', 'Berlin', 'DE'), ('Täubchenstraße', 'Berlin', 'DE'), ('Heiligenstadter Straße', 'Berlin', 'DE'), ('Afrikanische Straße', 'Berlin', 'DE'), ('Sleipnerstraße', 'Berlin', 'DE'), ('Alt-Lübars', 'Berlin', 'DE'), ('Niederkirchnerstraße', 'Berlin', 'DE'), ('Schendelgasse', 'Berlin', 'DE'), ('Gotenstraße', 'Berlin', 'DE'), ('Der Zwinger', 'Berlin', 'DE'), ('Thomasstraße', 'Berlin', 'DE'), ('Kurpro

In [41]:
df[df['city'] != 'Berlin']

,node,id,lat,long,country,city,street,woman,man
33711,node,1470618497,13.132996,52.387562,DE,Potsdam,Steinstraße,False,False
285929,node,3105084834,13.708112,52.395274,DE,Gosen-Neu Zittau,Bruchweg,False,False


In [46]:
bystreet.get_group(('Pestalozzistraße', 'Berlin', 'DE')).head()

,node,id,lat,long,country,city,street
2,node,26735763,13.320785,52.507339,DE,Berlin,Pestalozzistraße
2123,node,313012390,13.297754,52.507908,DE,Berlin,Pestalozzistraße
2124,node,313012391,13.298073,52.507899,DE,Berlin,Pestalozzistraße
2126,node,313012393,13.296902,52.507933,DE,Berlin,Pestalozzistraße
2127,node,313012394,13.296534,52.507943,DE,Berlin,Pestalozzistraße


1